## Task1 (Coding)

In this task, we will implement a full ML classifier based on decision trees (in python using Jupyter notebook). We will use the Mushroom Data Set to train and evaluate your classifier. This dataset comes from the UCI ML repository. (Hint: There are missing values in this dataset. At this particular time, you may ignore instances that have missing values and just remove them, or replace missing values with the mean value of the column. Please note that there are other ways of pre-processing data which we have not seen yet.)

You can use libraries e.g., Pandas, NumPy but you may NOT use any prebuilt decision tree packages. 

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 
  
# metadata 
print(mushroom.metadata) 
  
# variable information 
print(mushroom.variables) 


ConnectionError: Error connecting to server

Implement the basic decision tree procedure as discussed in the lectures. 
Implement 'DecisionTree' algorithm with a train procedure. 
Implement the information gain criterion as described in our lectures. 

In your report use one or two sentences to discuss the output (the output of the training procedure is the trained decision tree which is a representation of the if-then-else rules). 

You may print out your decision tree (you don't have to, however it might help you discuss the trained trees)  (This may be large - consider the best way to print it). 

In [18]:
import pandas as pd

left = pd.DataFrame(X)
right = pd.DataFrame(y)
# df = pd.concat([left, right])
df = pd.concat([left, right], axis=1)
df

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,poisonous
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


In [23]:
import pandas as pd
import numpy as np

class DecisionTree:
    def __init__(self):
        self.tree = None

    def fit(self, X, y):
        def entropy(labels):
            # Calculate entropy for a set of labels
            total_samples = len(labels)
            label_counts = np.array(list(Counter(labels).values()))
            probabilities = label_counts / total_samples
            return -np.sum(probabilities * np.log2(probabilities))

        def information_gain(parent_entropy, left_labels, right_labels):
            # Calculate Information Gain based on parent entropy and child entropies
            total_samples = len(left_labels) + len(right_labels)
            left_weight = len(left_labels) / total_samples
            right_weight = len(right_labels) / total_samples
            left_entropy = entropy(left_labels)
            right_entropy = entropy(right_labels)
            return parent_entropy - (left_weight * left_entropy) - (right_weight * right_entropy)

        def find_best_split(X, y):
            parent_entropy = entropy(y)
            best_feature, best_threshold, best_gain = None, None, -float('inf')
            for feature in X.columns:
                unique_values = np.unique(X[feature])
                for threshold in unique_values:
                    left_mask = X[feature] <= threshold
                    right_mask = ~left_mask
                    left_labels, right_labels = y[left_mask], y[right_mask]
                    gain = information_gain(parent_entropy, left_labels, right_labels)
                    if gain > best_gain:
                        best_gain, best_feature, best_threshold = gain, feature, threshold
            return best_feature, best_threshold

        def build_tree(X, y):
            if len(set(y)) == 1:
                return y[0]
            best_feature, best_threshold = find_best_split(X, y)
            left_mask = X[best_feature] <= best_threshold
            right_mask = ~left_mask
            left_tree = build_tree(X[left_mask], y[left_mask])
            right_tree = build_tree(X[right_mask], y[right_mask])
            return {'feature': best_feature, 'threshold': best_threshold,
                    'left': left_tree, 'right': right_tree}

        self.tree = build_tree(X, y)

    def predict(self, X):
        def predict_tree(node, sample):
            if isinstance(node, int):
                return node
            feature, threshold, left, right = node['feature'], node['threshold'], node['left'], node['right']
            if sample[feature] <= threshold:
                return predict_tree(left, sample)
            else:
                return predict_tree(right, sample)

        return [predict_tree(self.tree, sample) for _, sample in X.iterrows()]
    
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

tree_classifier = DecisionTree()
tree_classifier.fit(X_train, y_train)
y_pred = tree_classifier.predict(X_test)


NameError: name 'X_train' is not defined